In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi

Sun Feb 23 11:35:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               Off |   00000000:01:00.0 Off |                  Off |
| 42%   70C    P2            209W /  230W |    4289MiB /  24564MiB |     88%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import os

if 'notebooks' in os.getcwd():
    os.chdir("..")


from datasets import load_dataset, Dataset
from huggingface_hub import notebook_login
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import time
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import aiohttp
import asyncio
from tqdm.asyncio import tqdm
import torch
import scienceplots
plt.style.use(['science', 'no-latex'])
from IPython.display import clear_output

from src.text_utils import trim_text_to_token_limit
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

tqdm.pandas()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
torch.cuda.set_per_process_memory_fraction(0.6, device=0)


# Generate target summaries

In this notebook we will generate target summaries with a larger LM.

Let's start by loading everything into the machine

In [5]:
dataset = pd.read_json("data/wikipedia_dataset.json")

In [6]:
dataset.head()

,id,url,title,base_text,num_views,first_paragraph,predicted_topic,__index_level_0__,num_words,weight,word_weighting,num_tokens_base_text,text,num_tokens
4629,9513,https://pt.wikipedia.org/wiki/Conhecimento,Conhecimento,"Conhecimento (do latim cognoscere, ""ato de con...",129010,"Conhecimento (do latim cognoscere, ""ato de con...",politica e negócios,4656,2212,285370120,True,4010,"Conhecimento (do latim cognoscere, ""ato de con...",1785
691,1337,https://pt.wikipedia.org/wiki/Nazismo,Nazismo,"O nazismo (), oficialmente nacional-socialismo...",467240,"O nazismo (), oficialmente nacional-socialismo...",politica e negócios,697,10530,4920037200,True,19394,"O nazismo (), oficialmente nacional-socialismo...",2864
901,1718,https://pt.wikipedia.org/wiki/Sergipe,Sergipe,Sergipe é uma das 27 unidades federativas do B...,314672,Sergipe é uma das 27 unidades federativas do B...,politica e negócios,908,5172,1627483584,True,9891,Sergipe é uma das 27 unidades federativas do B...,3041
1465,3832,https://pt.wikipedia.org/wiki/Su%C3%AD%C3%A7a,Suíça,Suíça ( ; em suíço-alemão: Schwyz ou Schwiiz ;...,588196,Suíça ( ; em suíço-alemão: Schwyz ou Schwiiz ;...,politica e negócios,1475,9452,5559628592,True,17521,Suíça ( ; em suíço-alemão: Schwyz ou Schwiiz ;...,674
48,302,https://pt.wikipedia.org/wiki/Arist%C3%B3teles,Aristóteles,"Aristóteles (; Estagira, – Atenas, ) foi um f...",348607,"Aristóteles (; Estagira, – Atenas, ) foi um f...",politica e negócios,49,7075,2466394525,True,12841,"Aristóteles (; Estagira, – Atenas, ) foi um f...",2666


In [7]:
model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"



model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name, # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = 6_000,
    dtype = None,
    load_in_4bit = True,
    fast_inference=True,
    cache_dir = '/Data'
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

INFO 02-23 11:35:44 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA RTX A5000. Max memory: 23.651 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-bnb-4bit with actual GPU utilization = 41.12%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 23.65 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 6000. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 3.39 GB. Also swap space = 6 GB.
INFO 02-23 11:35:53 config.py:549] This model supports multiple tasks: {'embed', 'generate', 'classify', 'score', 'reward'}. 

[W223 11:35:55.551062277 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 02-23 11:35:55 weight_utils.py:254] Using model weights format ['*.safetensors']


model.safetensors:  64%|######4   | 3.66G/5.70G [00:00<?, ?B/s]

INFO 02-23 11:36:50 weight_utils.py:270] Time spent downloading weights for unsloth/meta-llama-3.1-8b-instruct-bnb-4bit: 54.171182 seconds


...

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-23 11:37:53 model_runner.py:1115] Loading model weights took 5.3229 GB
INFO 02-23 11:37:55 worker.py:267] Memory profiling takes 1.68 seconds
INFO 02-23 11:37:55 worker.py:267] the current vLLM instance can use total_gpu_memory (23.65GiB) x gpu_memory_utilization (0.41) = 9.73GiB
INFO 02-23 11:37:55 worker.py:267] model weights take 5.32GiB; non_torch_memory takes 0.00GiB; PyTorch activation peak memory takes 0.77GiB; the rest of the memory reserved for KV Cache is 3.63GiB.
INFO 02-23 11:37:55 executor_base.py:111] # cuda blocks: 1859, # CPU blocks: 3072
INFO 02-23 11:37:55 executor_base.py:116] Maximum concurrency for 6000 tokens per request: 4.96x
INFO 02-23 11:37:57 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utiliz

Capturing CUDA graph shapes: 100%|██████████| 18/18 [00:13<00:00,  1.29it/s]

INFO 02-23 11:38:11 model_runner.py:1562] Graph capturing finished in 14 secs, took -3.55 GiB
INFO 02-23 11:38:11 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 18.64 seconds


In [14]:
base_prompt = '''
    Você é um assistente virtual que deve gerar resumos de textos em português. 
    Seu resumo deve ter, no maximo 200 palavras e conter todas as informações principais do texto.
    Esse é o texto:

    {text}

    Faça um resumo de no maximo 200 palavras do texto acima
'''

In [ ]:


tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

FastLanguageModel.for_inference(model)

generated = []

for _, row in tqdm(dataset.sort_values("num_tokens", ascending= False).iterrows(), total = 5000):
    prompt = base_prompt.format(text = row["text"])
    message = [{'role': 'user', 'content': prompt}]
    inputs = tokenizer.apply_chat_template(
        message,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")

    generated_ids = model.generate(inputs, max_new_tokens = 500)

    generated_text = tokenizer.decode(generated_ids[0, inputs.shape[1]:])

    generated.append({
        'id': row['id'],
        'prompt': prompt,
        'generated_text': generated_text
    })
    # break
    clear_output()
    print(generated_text)

In [17]:
pd.DataFrame(generated)

,id,prompt,generated_text
0,57237,\n Você é um assistente virtual que deve ge...,O Los Angeles Lakers é um time de basquetebol ...
1,10306,\n Você é um assistente virtual que deve ge...,O elétron é uma partícula subatômica com carga...
2,2118886,\n Você é um assistente virtual que deve ge...,"Sofia Doroteia Ulrica Alice, rainha consorte d..."
3,1695,\n Você é um assistente virtual que deve ge...,Um sistema operacional é um programa ou conjun...
4,5448408,\n Você é um assistente virtual que deve ge...,A crise migratória venezuelana refere-se à emi...
5,1232,\n Você é um assistente virtual que deve ge...,A monarquia é uma forma de governo em que um m...
6,12344,\n Você é um assistente virtual que deve ge...,A Igreja Católica é a maior igreja cristã do m...
7,115343,\n Você é um assistente virtual que deve ge...,O Athlon é uma série de processadores da plata...
8,13675,\n Você é um assistente virtual que deve ge...,Alfred Joseph Hitchcock foi um diretor e produ...
9,7112,\n Você é um assistente virtual que deve ge...,Titã é o maior satélite natural de Saturno e o...
